# Auto Merging Retriever

In this notebook, we showcase our `AutoMergingRetriever`, which looks at a set of leaf nodes and recursively "merges" subsets of leaf nodes that reference a parent node beyond a given threshold. This allows us to consolidate potentially disparate, smaller contexts into a larger context that might help synthesis.

You can define this hierarchy yourself over a set of documents, or you can make use of our brand-new text parser: a HierarchicalNodeParser that takes in a candidate set of documents and outputs an entire hierarchy of nodes, from "coarse-to-fine".

In [16]:
import os 
from dotenv import load_dotenv, find_dotenv

In [17]:
load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')

True

In [18]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

## Load Data

Let's first load the Llama 2 paper: https://arxiv.org/pdf/2307.09288.pdf. This will be our test data.

In [1]:
!mkdir -p 'data/'
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "data/llama2.pdf"

--2024-03-31 01:22:20--  https://arxiv.org/pdf/2307.09288.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.195.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13661300 (13M) [application/pdf]
Saving to: ‘data/llama2.pdf’

data/llama2.pdf     100%[===================>]  13.03M  2.34MB/s    in 11s     

2024-03-31 01:22:31 (1.24 MB/s) - ‘data/llama2.pdf’ saved [13661300/13661300]



In [4]:
from pathlib import Path
from llama_index.readers.file import PyMuPDFReader

In [1]:
from llama_index.core import SimpleDirectoryReader

In [4]:
docs0 = SimpleDirectoryReader(input_files=["./data/llama2.pdf"]).load_data()

In [5]:
loader = PyMuPDFReader()
# docs0 = loader.load_data(file=Path("./data/llama2.pdf"))
docs0 = loader.load(file_path=Path("./data/llama2.pdf"))

By default, the PDF reader creates a separate doc for each page.
For the sake of this notebook, we stitch docs together into one doc.
This will help us better highlight auto-merging capabilities that "stitch" chunks together later on.

In [5]:
from llama_index.core import Document

doc_text = "\n\n".join([d.get_content() for d in docs0])
docs = [Document(text=doc_text)]

## Parse Chunk Hierarchy from Text, Load into Storage

In this section we make use of the `HierarchicalNodeParser`. This will output a hierarchy of nodes, from top-level nodes with bigger chunk sizes to child nodes with smaller chunk sizes, where each child node has a parent node with a bigger chunk size.

By default, the hierarchy is:
- 1st level: chunk size 2048
- 2nd level: chunk size 512
- 3rd level: chunk size 128


We then load these nodes into storage. The leaf nodes are indexed and retrieved via a vector store - these are the nodes that will first be directly retrieved via similarity search. The other nodes will be retrieved from a docstore.

In [6]:
from llama_index.core.node_parser import HierarchicalNodeParser, SentenceSplitter

In [7]:
node_parser = HierarchicalNodeParser.from_defaults()

In [8]:
nodes = node_parser.get_nodes_from_documents(docs)

In [9]:
len(nodes)

1015

Here we import a simple helper function for fetching "leaf" nodes within a node list.
These are nodes that don't have children of their own.

In [10]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [11]:
leaf_nodes = get_leaf_nodes(nodes)

In [19]:
len(leaf_nodes)

793

In [20]:
root_nodes = get_root_nodes(nodes)

### Load into Storage

We define a docstore, which we load all nodes into.

We then define a `VectorStoreIndex` containing just the leaf-level nodes.

In [21]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage import StorageContext
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore)

In [22]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    embed_model = OpenAIEmbedding(model='text-embedding-3-small'),
    llm = OpenAI(model="gpt-3.5-turbo"),
    storage_context=storage_context,
)

## Define Retriever

In [23]:
from llama_index.core.retrievers.auto_merging_retriever import AutoMergingRetriever

In [24]:
base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [25]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "What could be the potential outcomes of adjusting the amount of safety data used in the RLHF stage?"
)

base_nodes = base_retriever.retrieve(query_str)

nodes = retriever.retrieve(query_str)

> Merging 3 nodes into parent node.
> Parent node id: 17ef2f43-9553-4dca-bc57-e22127535a59.
> Parent node text: We conduct RLHF by first collecting human preference data for safety similar to Section 3.2.2: an...



In [25]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "What could be the potential outcomes of adjusting the amount of safety data used in the RLHF stage?"
)

base_nodes = base_retriever.retrieve(query_str)

nodes = retriever.retrieve(query_str)

In [26]:
len(nodes)

6

In [27]:
len(base_nodes)

6

In [31]:
len(nodes)

4

In [32]:
len(base_nodes)

6

In [33]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)

**Node ID:** a57425b5-8a5b-410e-b3a9-e172a6691787<br>**Similarity:** 0.6427251889885033<br>**Text:** We also list two
qualitative examples where safety and helpfulness reward models don’t agree with each other in Table 35.
A.4.2
Qualitative Results on Safety Data Scaling
In Section 4.2.3, we study the impact of adding more safety data into model RLHF in a quantitative manner.
Here we showcase a few samples to qualitatively examine the evolution of model behavior when we scale
safety data in Tables 36, 37, and 38.<br>

**Node ID:** e327434b-c4eb-49a1-a82c-563f1d695f88<br>**Similarity:** 0.6249924876358756<br>**Text:** We conduct RLHF by first collecting human preference data for safety similar to Section 3.2.2: annotators
write a prompt that they believe can elicit unsafe behavior, and then compare multiple model responses to
the prompts, selecting the response that is safest according to a set of guidelines. We then use the human
preference data to train a safety reward model (see Section 3.2.2), and also reuse the adversarial prompts to
sample from the model during the RLHF stage.
Better Long-Tail Safety Robustness without Hurting Helpfulness
Safety is inherently a long-tail problem,
where the challenge comes from a small number of very specific cases. We investigate the impact of Safety
RLHF by taking two intermediate Llama 2-Chat checkpoints—one without adversarial prompts in the RLHF
stage and one with them—and score their responses on our test sets using our safety and helpfulness reward
models. In Figure 14, we plot the score distribution shift of the safety RM on the safety test set (left) and that
of the helpfulness RM on the helpfulness test set (right). In the left hand side of the figure, we observe that
the distribution of safety RM scores on the safety set shifts to higher reward scores after safety tuning with
RLHF, and that the long tail of the distribution near zero thins out. A clear cluster appears on the top-left
corner suggesting the improvements of model safety. On the right side, we do not observe any gathering
pattern below the y = x line on the right hand side of Figure 14, which indicates that the helpfulness score
distribution is preserved after safety tuning with RLHF. Put another way, given sufficient helpfulness training
data, the addition of an additional stage of safety mitigation does not negatively impact model performance
on helpfulness to any notable degradation. A qualitative example is shown in Table 12.
Impact of Safety Data Scaling.
A tension between helpfulness and safety of LLMs has been observed in
previous studies (Bai et al., 2022a). To better understand how the addition of safety training data affects
general model performance, especially helpfulness, we investigate the trends in safety data scaling by
adjusting the amount of safety data used in the RLHF stage.<br>

**Node ID:** 8a40bd31-7ec1-4e9a-b52e-a076ac6775c2<br>**Similarity:** 0.605330852334444<br>**Text:** This teaches
the model to align with our safety guidelines even before RLHF, and thus lays the foundation for
high-quality human preference data annotation.
2. Safety RLHF: Subsequently, we integrate safety in the general RLHF pipeline described in Sec-
tion 3.2.2. This includes training a safety-specific reward model and gathering more challenging
adversarial prompts for rejection sampling style fine-tuning and PPO optimization.
3. Safety Context Distillation: Finally, we refine our RLHF pipeline with context distillation (Askell
et al., 2021b).<br>

**Node ID:** db926842-50f2-413f-90de-9210bd48c15d<br>**Similarity:** 0.5590218161526865<br>**Text:** 10%
Safety Data Pct. 25%
Safety Data Pct. 50%
0.0
0.2
0.4
0.6
0.8
1.0
Safety Reward Model Score
Safety Data Pct. 100%
Figure 15: Safety data scaling trends. Left: as we increase the amount of safety data in model training, the
mean safety RM score improves significantly while the helpfulness counterpart remains relatively stable.
Right: the left tail of safety RM scores (i.e., most unsafe responses) gradually disappears with the addition of
more safety training data.<br>

In [34]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** 236c2d06-5446-4b39-a803-3e34d303db59<br>**Similarity:** 0.6626584461481428<br>**Text:** A qualitative example is shown in Table 12.
Impact of Safety Data Scaling.
A tension between helpfulness and safety of LLMs has been observed in
previous studies (Bai et al., 2022a). To better understand how the addition of safety training data affects
general model performance, especially helpfulness, we investigate the trends in safety data scaling by
adjusting the amount of safety data used in the RLHF stage.<br>

**Node ID:** a57425b5-8a5b-410e-b3a9-e172a6691787<br>**Similarity:** 0.6426103501546494<br>**Text:** We also list two
qualitative examples where safety and helpfulness reward models don’t agree with each other in Table 35.
A.4.2
Qualitative Results on Safety Data Scaling
In Section 4.2.3, we study the impact of adding more safety data into model RLHF in a quantitative manner.
Here we showcase a few samples to qualitatively examine the evolution of model behavior when we scale
safety data in Tables 36, 37, and 38.<br>

**Node ID:** c0ad92af-56c4-4389-a52b-1b37154b653b<br>**Similarity:** 0.63198309607627<br>**Text:** A clear cluster appears on the top-left
corner suggesting the improvements of model safety. On the right side, we do not observe any gathering
pattern below the y = x line on the right hand side of Figure 14, which indicates that the helpfulness score
distribution is preserved after safety tuning with RLHF. Put another way, given sufficient helpfulness training
data, the addition of an additional stage of safety mitigation does not negatively impact model performance
on helpfulness to any notable degradation. A qualitative example is shown in Table 12.
Impact of Safety Data Scaling.<br>

**Node ID:** 8a40bd31-7ec1-4e9a-b52e-a076ac6775c2<br>**Similarity:** 0.6052545282623665<br>**Text:** This teaches
the model to align with our safety guidelines even before RLHF, and thus lays the foundation for
high-quality human preference data annotation.
2. Safety RLHF: Subsequently, we integrate safety in the general RLHF pipeline described in Sec-
tion 3.2.2. This includes training a safety-specific reward model and gathering more challenging
adversarial prompts for rejection sampling style fine-tuning and PPO optimization.
3. Safety Context Distillation: Finally, we refine our RLHF pipeline with context distillation (Askell
et al., 2021b).<br>

**Node ID:** 5efa6865-e59c-40d2-ad34-e85ac3b2d00e<br>**Similarity:** 0.5801062608757874<br>**Text:** We conduct RLHF by first collecting human preference data for safety similar to Section 3.2.2: annotators
write a prompt that they believe can elicit unsafe behavior, and then compare multiple model responses to
the prompts, selecting the response that is safest according to a set of guidelines. We then use the human
preference data to train a safety reward model (see Section 3.2.2), and also reuse the adversarial prompts to
sample from the model during the RLHF stage.<br>

**Node ID:** db926842-50f2-413f-90de-9210bd48c15d<br>**Similarity:** 0.5589974682888499<br>**Text:** 10%
Safety Data Pct. 25%
Safety Data Pct. 50%
0.0
0.2
0.4
0.6
0.8
1.0
Safety Reward Model Score
Safety Data Pct. 100%
Figure 15: Safety data scaling trends. Left: as we increase the amount of safety data in model training, the
mean safety RM score improves significantly while the helpfulness counterpart remains relatively stable.
Right: the left tail of safety RM scores (i.e., most unsafe responses) gradually disappears with the addition of
more safety training data.<br>

## Plug it into Query Engine

In [35]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [36]:
query_engine = RetrieverQueryEngine.from_args(retriever)
base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

In [37]:
response = query_engine.query(query_str)

> Merging 3 nodes into parent node.
> Parent node id: e327434b-c4eb-49a1-a82c-563f1d695f88.
> Parent node text: We conduct RLHF by first collecting human preference data for safety similar to Section 3.2.2: an...



In [38]:
print(str(response))

The potential outcomes of adjusting the amount of safety data used in the RLHF stage could include an improvement in the mean safety reward model score, with the helpfulness counterpart remaining relatively stable. Additionally, as more safety data is incorporated into model training, the left tail of safety reward model scores, representing the most unsafe responses, may gradually disappear.


In [39]:
base_response = base_query_engine.query(query_str)

In [40]:
print(str(base_response))

Adjusting the amount of safety data used in the RLHF stage could potentially lead to improvements in model safety performance without significantly impacting the helpfulness score distribution. It may help the model align with safety guidelines early on, laying a foundation for high-quality human preference data annotation. Additionally, increasing the amount of safety data in model training could result in a significant improvement in the mean safety reward model score while keeping the helpfulness counterpart relatively stable. Furthermore, the addition of more safety training data may gradually eliminate the most unsafe responses, as indicated by the disappearance of the left tail of safety reward model scores.


## Evaluation

We evaluate how well the hierarchical retriever works compared to the baseline retriever in a more quantitative manner.

**WARNING**: This can be *expensive*, especially with GPT-4. Use caution and tune the sample size to fit your budget.

In [41]:
from llama_index.core.evaluation import DatasetGenerator, QueryResponseDataset
from llama_index.llms.openai import OpenAI
import nest_asyncio

nest_asyncio.apply()

In [38]:
gpt4 = OpenAI(model='gpt-4o')

In [ ]:
# NOTE: run this if the dataset isn't already saved
dataset_generator = DatasetGenerator(
    root_nodes[:20],
    llm=gpt4,
    show_progress=True,
    num_questions_per_chunk=3,
)

In [ ]:
eval_dataset = await dataset_generator.agenerate_dataset_from_nodes(num=60)

In [ ]:
eval_dataset.save_json("data/llama2_eval_qr_dataset.json")

In [ ]:
# optional
eval_dataset = QueryResponseDataset.from_json(
    "data/llama2_eval_qr_dataset.json"
)

### Compare Results

We run evaluations on each of the retrievers: correctness, semantic similarity, relevance, and faithfulness.

In [ ]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [30]:
from llama_index.core.evaluation import (
    CorrectnessEvaluator,
    SemanticSimilarityEvaluator,
    RelevancyEvaluator,
    FaithfulnessEvaluator,
    PairwiseComparisonEvaluator,
)


from collections import defaultdict
import pandas as pd

In [31]:
gpt4 = OpenAI(temperature=0, model="gpt-4o")

In [33]:
# NOTE: can uncomment other evaluators
evaluator_c = CorrectnessEvaluator(llm=gpt4)
evaluator_s = SemanticSimilarityEvaluator(embed_model=OpenAIEmbedding(model='text-embedding-3-small'))
evaluator_r = RelevancyEvaluator(llm=gpt4)
evaluator_f = FaithfulnessEvaluator(llm=gpt4)
# pairwise_evaluator = PairwiseComparisonEvaluator(service_context=eval_service_context)

In [34]:
from llama_index.core.evaluation.eval_utils import get_responses, get_results_df
from llama_index.core.evaluation import BatchEvalRunner

In [36]:
eval_results = await batch_runner.aevaluate_response_strs(
    eval_qs, responses=pred_responses, reference=ref_response_strs
)

NameError: name 'eval_qs' is not defined

In [37]:
eval_qs = eval_dataset.questions
qr_pairs = eval_dataset.qr_pairs
ref_response_strs = [r for (_, r) in qr_pairs]

NameError: name 'eval_dataset' is not defined

In [ ]:
pred_responses = get_responses(eval_qs, query_engine, show_progress=True)

In [ ]:
base_pred_responses = get_responses(
    eval_qs, base_query_engine, show_progress=True
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:07<00:00,  8.17it/s]


In [ ]:
import numpy as np

pred_response_strs = [str(p) for p in pred_responses]
base_pred_response_strs = [str(p) for p in base_pred_responses]

In [35]:
evaluator_dict = {
    "correctness": evaluator_c,
    "faithfulness": evaluator_f,
    "relevancy": evaluator_r,
    "semantic_similarity": evaluator_s,
}
batch_runner = BatchEvalRunner(evaluator_dict, workers=2, show_progress=True)

In [ ]:
eval_results = await batch_runner.aevaluate_responses(
    eval_qs, responses=pred_responses, reference=ref_response_strs
)

In [ ]:
base_eval_results = await batch_runner.aevaluate_responses(
    eval_qs, responses=base_pred_responses, reference=ref_response_strs
)

In [ ]:
results_df = get_results_df(
    [eval_results, base_eval_results],
    ["Auto Merging Retriever", "Base Retriever"],
    ["correctness", "relevancy", "faithfulness", "semantic_similarity"],
)
display(results_df)

,names,correctness,relevancy,faithfulness,semantic_similarity
0,Auto Merging Retriever,4.266667,0.916667,0.95,0.962196
1,Base Retriever,4.208333,0.916667,0.95,0.960602


**Analysis**: The results are roughly the same.

Let's also try to see which answer GPT-4 prefers with our pairwise evals.

In [ ]:
batch_runner = BatchEvalRunner(
    {"pairwise": pairwise_evaluator}, workers=10, show_progress=True
)

In [ ]:
pairwise_eval_results = await batch_runner.aevaluate_response_strs(
    eval_qs,
    response_strs=pred_response_strs,
    reference=base_pred_response_strs,
)
pairwise_score = np.array(
    [r.score for r in pairwise_eval_results["pairwise"]]
).mean()

In [ ]:
pairwise_score

0.525

**Analysis**: The pairwise comparison score is a measure of the percentage of time the candidate answer (using auto-merging retriever) is preferred vs. the base answer (using the base retriever). Here we see that it's roughly even.